In [1]:
import pandas as pd
import numpy as np
import seaborn as snb
import os
import pickle
import sklearn as sk
import sklearn.ensemble as ske
from sklearn import model_selection, tree, linear_model,svm
from sklearn.feature_selection import SelectFromModel
import joblib
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
import operator
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier


In [2]:
#print(os.listdir())

In [3]:
df=pd.read_csv("data.csv", sep="|")

In [4]:
df



,Name,md5,Machine,SizeOfOptionalHeader,Characteristics,MajorLinkerVersion,MinorLinkerVersion,SizeOfCode,SizeOfInitializedData,SizeOfUninitializedData,...,ResourcesNb,ResourcesMeanEntropy,ResourcesMinEntropy,ResourcesMaxEntropy,ResourcesMeanSize,ResourcesMinSize,ResourcesMaxSize,LoadConfigurationSize,VersionInformationSize,legitimate
0,memtest.exe,631ea355665f28d4707448e442fbf5b8,332,224,258,9,0,361984,115712,0,...,4,3.262823,2.568844,3.537939,8797.000000,216,18032,0,16,1
1,ose.exe,9d10f99a6712e28f8acd5641e3a7ea6b,332,224,3330,9,0,130560,19968,0,...,2,4.250461,3.420744,5.080177,837.000000,518,1156,72,18,1
2,setup.exe,4d92f518527353c0db88a70fddcfd390,332,224,3330,9,0,517120,621568,0,...,11,4.426324,2.846449,5.271813,31102.272727,104,270376,72,18,1
3,DW20.EXE,a41e524f8d45f0074fd07805ff0c9b12,332,224,258,9,0,585728,369152,0,...,10,4.364291,2.669314,6.400720,1457.000000,90,4264,72,18,1
4,dwtrig20.exe,c87e561258f2f8650cef999bf643a731,332,224,258,9,0,294912,247296,0,...,2,4.306100,3.421598,5.190603,1074.500000,849,1300,72,18,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138042,VirusShare_8e292b418568d6e7b87f2a32aee7074b,8e292b418568d6e7b87f2a32aee7074b,332,224,258,11,0,205824,223744,0,...,7,4.122736,1.370260,7.677091,14900.714286,16,81654,72,0,0
138043,VirusShare_260d9e2258aed4c8a3bbd703ec895822,260d9e2258aed4c8a3bbd703ec895822,332,224,33167,2,25,37888,185344,0,...,26,3.377663,2.031619,5.050074,6905.846154,44,67624,0,15,0
138044,VirusShare_8d088a51b7d225c9f5d11d239791ec3f,8d088a51b7d225c9f5d11d239791ec3f,332,224,258,10,0,118272,380416,0,...,22,6.825406,2.617026,7.990487,14981.909091,48,22648,72,14,0
138045,VirusShare_4286dccf67ca220fe67635388229a9f3,4286dccf67ca220fe67635388229a9f3,332,224,33166,2,25,49152,16896,0,...,10,3.421627,2.060964,4.739744,601.600000,16,2216,0,0,0


In [5]:
X = df.drop(['Name', 'md5', 'legitimate'], axis=1).values
y = df['legitimate'].values


In [6]:
X
y
X.shape[1]

54

In [7]:
featureSel = ske.ExtraTreesClassifier().fit(X, y)
model = SelectFromModel(featureSel, prefit=True)
X_new = model.transform(X)
nb_features = X_new.shape[1]

In [8]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X_new, y ,test_size=0.2)

In [9]:
features = []
nb_features #imp features

13

In [10]:
indices = np.argsort(featureSel.feature_importances_)[::-1][:nb_features]

In [11]:
indices

array([24,  0, 53,  2, 23, 11, 34,  1, 18, 48, 47, 33, 14])

In [12]:
for f in range(nb_features):
    print("%d. feature %s (%f)" % (f + 1, df.columns[2+indices[f]], featureSel.feature_importances_[indices[f]]))

1. feature DllCharacteristics (0.153084)
2. feature Machine (0.093354)
3. feature VersionInformationSize (0.083009)
4. feature Characteristics (0.081107)
5. feature Subsystem (0.064561)
6. feature ImageBase (0.064230)
7. feature SectionsMaxEntropy (0.054443)
8. feature SizeOfOptionalHeader (0.050382)
9. feature MajorSubsystemVersion (0.048081)
10. feature ResourcesMaxEntropy (0.039746)
11. feature ResourcesMinEntropy (0.025773)
12. feature SectionsMinEntropy (0.021050)
13. feature MajorOperatingSystemVersion (0.020728)


In [13]:
for f in sorted(np.argsort(featureSel.feature_importances_)[::-1][:nb_features]):
    features.append(df.columns[2+f])

In [21]:
features

['Machine',
 'SizeOfOptionalHeader',
 'Characteristics',
 'ImageBase',
 'MajorOperatingSystemVersion',
 'MajorSubsystemVersion',
 'Subsystem',
 'DllCharacteristics',
 'SectionsMinEntropy',
 'SectionsMaxEntropy',
 'ResourcesMinEntropy',
 'ResourcesMaxEntropy',
 'VersionInformationSize']

In [22]:
models = {
    #"LR" : linear_model.LogisticRegression(),
    "KNN": KNeighborsClassifier(),
    "DecisionTree": tree.DecisionTreeClassifier(),
    "RandomForest": ske.RandomForestClassifier(),
    "GradientBoosting": ske.GradientBoostingClassifier(),
    "AdaBoost": ske.AdaBoostClassifier(),
    #"GNB": GaussianNB(),
    #"SVC": svm.SVC()
    }

In [23]:
results = {}
print("\nNow testing algorithms")
for x in models:
    clf = models[x]
    clf.fit(X_train, y_train)
    score = clf.score(X_test, y_test)
    print("%s : %f %%" % (x, score*100))
    results[x] = score


Now testing algorithms
KNN : 98.953278 %
DecisionTree : 99.137993 %
RandomForest : 99.373415 %
GradientBoosting : 98.942412 %
AdaBoost : 98.515031 %


In [29]:
winner = max(results, key=results.get)
print('\nWinner algorithm is %s with a %f %% success' % (winner, results[winner]*100))


Winner algorithm is RandomForest with a 99.373415 % success


In [30]:
print('Saving algorithm and feature list in classifier directory...')
joblib.dump(results[winner], 'classifier/classifier.pkl')
open('classifier/features.pkl', 'w').write(pickle.dumps(features))
print('Saved')

Saving algorithm and feature list in classifier directory...


TypeError: write() argument must be str, not bytes

In [31]:
clf = results[winner]
clf.fit(X_train, Y_train)
res = clf.predict(X_test)
mt = confusion_matrix(y_test, res)
print("False positive rate : %f %%" % ((mt[0][1] / float(sum(mt[0])))*100))
print('False negative rate : %f %%' % ( (mt[1][0] / float(sum(mt[1]))*100)))

AttributeError: 'numpy.float64' object has no attribute 'fit'